In [1]:
from dataclasses import dataclass
import os
import json
import pandas
import string
import random
import csv
import math


In [2]:
# cartella dei csv
csv_dir = 'Crowd_frame/data/result/secondo_progetto_social_computing/Dataframe/'

def serialize_json(folder, filename, data):
    # Se la cartella non esiste la creo
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

# Lettura file JSON da locale
def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file, object_hook = lambda d: {int(k) if k.lstrip('-').isdigit() else k: v for k, v in d.items()}) # object_hook per convertire le stringhe numeriche in numeri (es. id del follower)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

def id_generator(size=11, chars=string.ascii_uppercase):
    return ''.join(random.choice(chars) for _ in range(size))

# Controllo la riga in cui sto inserendo l'elemento per evitare di inserire elementi provenienti dalla stessa riga del csv
def check_row(hits, random_row, x):
    for i in range(3):
        if(hits[random_row][i] != 0 and hits[random_row][i].type[:1] == 'G'):
            if(hits[random_row][(i+1)%3] != 0 and hits[random_row][(i+1)%3].type[:1] == 'E'):
                if(int(hits[random_row][(i+1)%3].type[1:]) - 1 == x):
                    return False
            if(hits[random_row][(i+2)%3] != 0 and hits[random_row][(i+2)%3].type[:1] == 'E'):
                if(int(hits[random_row][(i+2)%3].type[1:]) - 1 == x):
                    return False
            else:
                return True

# Funzione per il percent agreemnt del Explanation quality
def in_same_interval(values, ground, compared, n):
    Lie = range(0,19)
    false = range(20,35)
    Barely_True = range(36,50)
    Half_true = range(51,65)
    Mostly_true = range(66,80)
    true = range(81,100)
    
    if(ground in Lie and compared in Lie):
        values.loc['Statement ' + str(n), 'Explanation quality'] += 1
    elif(ground in false and compared in false):
        values.loc['Statement ' + str(n), 'Explanation quality'] += 1
    elif(ground in Barely_True and compared in Barely_True):
        values.loc['Statement ' + str(n), 'Explanation quality'] += 1
    elif(ground in Half_true and compared in Half_true):
        values.loc['Statement ' + str(n), 'Explanation quality'] += 1
    elif(ground in Mostly_true and compared in Mostly_true):
        values.loc['Statement ' + str(n), 'Explanation quality'] += 1
    elif(ground in true and compared in true):
        values.loc['Statement ' + str(n), 'Explanation quality'] += 1

# funione per calcolare la media delle dimensioni qualitative delle explanation
def average_explanation_quality_values (average_explanation_values, explanation, string):
    for i in range (len(explanation.index)):
        average_explanation_values.loc['Truthfulness 1', 'Explanation ' + string] += workers_answers.loc[explanation.index.values[i], 'doc_truthfulness1_value']
        average_explanation_values.loc['Truthfulness 2', 'Explanation ' + string] += workers_answers.loc[explanation.index.values[i], 'doc_truthfulness2_value']
        average_explanation_values.loc['Explanation quality', 'Explanation ' + string] += workers_answers.loc[explanation.index.values[i], 'doc_explanationquality_value']

    average_explanation_values.loc['Truthfulness 1', 'Explanation ' + string] /= len(explanation.index)
    average_explanation_values.loc['Truthfulness 2', 'Explanation ' + string] /= len(explanation.index)
    average_explanation_values.loc['Explanation quality', 'Explanation ' + string] /= len(explanation.index)

# funione per calcolare la media della lunghezza delle note divise per explanation
def average_text_percentage(text_percentage, explanation, string):

    for i in range (len(explanation.index)):
        text_percentage.loc['percentage', 'Explanation ' + string] += len(workers_notes.loc[explanation.index.values[i], 'note_text_current'])

    text_percentage.loc['percentage', 'Explanation ' + string] /= len(explanation.index)

    if(len(explanation) > 0):
        text_percentage.loc['percentage', 'Explanation ' + string] /= len(workers_notes.loc[explanation.index.values[0], 'note_text_right']) + workers_notes.loc[explanation.index.values[0], 'note_index_end']

# funzione per assegnare explanation, statement, label e id corretti in base alle note, alcuni
# worker avevano dati riguardanti gli elementi delle HITs in workers_answers che non combaciavano 
# con i dati in workers_notes, quindi siamo risaliti alle explanation, statement, label e id corretti 
# grazie alle note   
def answers_fix(n, x, new_workers_answers, new_workers_notes, fever):
    while(n < x):
        explanation = new_workers_notes.loc[n, 'note_text_left'] + new_workers_notes.loc[n, 'note_text_raw'] + new_workers_notes.loc[n, 'note_text_right']      
        if(explanation == fever.loc[0, 'explanation_human']):            
            new_workers_answers.loc[n, 'doc_id'] = 183468
            new_workers_answers.loc[n, 'doc_statement'] = fever.loc[0, 'statement']
            new_workers_answers.loc[n, 'doc_explanation'] = fever.loc[0, 'explanation_human']
            new_workers_answers.loc[n, 'doc_label'] = fever.loc[0, 'label']
        elif(explanation == fever.loc[1, 'explanation_human']):            
            new_workers_answers.loc[n, 'doc_id'] = 185415
            new_workers_answers.loc[n, 'doc_statement'] = fever.loc[1, 'statement']
            new_workers_answers.loc[n, 'doc_explanation'] = fever.loc[1, 'explanation_human']
            new_workers_answers.loc[n, 'doc_label'] = fever.loc[1, 'label']
        elif(explanation == fever.loc[2, 'explanation_human']):            
            new_workers_answers.loc[n, 'doc_id'] = 63527
            new_workers_answers.loc[n, 'doc_statement'] = fever.loc[2, 'statement']
            new_workers_answers.loc[n, 'doc_explanation'] = fever.loc[2, 'explanation_human']
            new_workers_answers.loc[n, 'doc_label'] = fever.loc[2, 'label']
        elif(explanation == fever.loc[0, 'explanation_model']):            
            new_workers_answers.loc[n, 'doc_id'] = 183468
            new_workers_answers.loc[n, 'doc_statement'] = fever.loc[0, 'statement']
            new_workers_answers.loc[n, 'doc_explanation'] = fever.loc[0, 'explanation_model']
            new_workers_answers.loc[n, 'doc_label'] = fever.loc[0, 'label']
        elif(explanation == fever.loc[1, 'explanation_model']):            
            new_workers_answers.loc[n, 'doc_id'] = 185415
            new_workers_answers.loc[n, 'doc_statement'] = fever.loc[1, 'statement']
            new_workers_answers.loc[n, 'doc_explanation'] = fever.loc[1, 'explanation_model']
            new_workers_answers.loc[n, 'doc_label'] = fever.loc[1, 'label']
        elif(explanation == fever.loc[2, 'explanation_model']):            
            new_workers_answers.loc[n, 'doc_id'] = 63527
            new_workers_answers.loc[n, 'doc_statement'] = fever.loc[2, 'statement']
            new_workers_answers.loc[n, 'doc_explanation'] = fever.loc[2, 'explanation_model']
            new_workers_answers.loc[n, 'doc_label'] = fever.loc[2, 'label']
        n += 1


# Dataclass per gestire gli elementi degli HITs
@dataclass

class Elemento:
    id: int
    statement: str
    explanation: str
    label: str
    type: str # E1,E2,E3 o G1,G2,G3

def _init_(self, id: int, statement: str, explanation: str, label: str, type: str):
    self.id = id
    self.statement = statement
    self.explanation = explanation
    self.label = label
    self.type = type


In [3]:
fever = pandas.read_csv('group_7-fabris_chivella_strazzi.csv')


# 3.Scrivete un Jupyter Notebook al fine di creare gli HITs del vostro task a partire dal dataset ricevuto, con le seguenti caratteristiche:
1. Ogni HIT contiene 3 elementi
2. Ogni elemento di un HIT ha 4 attributi:
    - id:identificatoredellostatement
    - statement:testodellostatement
    - explanation:testodellaspiegazione
    - label:etichettadellaspiegazione
3. Due elementi su tre (2/3) di un HIT assegnano all’attributo explanation il valore di explanation_model
4. Un elemento su tre (1/3) di un HIT assegnano all’attributo explanation il valore di explanation_human
5. Ogni valore di explanation_model dev’essere valutato da 3 worker diversi
6. Ogni valore di explanation_human dev’essere valutato da 2 worker diversi
7. La posizione dei 3 elementi in un HIT deve essere casuale
8. ll numero totale di HIT è pari a 12
Si supponga che E1, E2, E3 indichino le spiegazioni generate, mentre G1, G2 e G3 quelle fornite da esseri umani. La seguente tabella mostra un possibile schema valido di allocazione di sei dei dodici HITs richiesti:
| Posizione 1 | Posizione 2 | Posizione 3 |
| ------------------ | ------------------ | ------------------ |
| E1 | G1 | E2 |
| G1 | E3 | E1 |
| E2 | E1 | G2 |
| E2 | G2 | E3 |
| E3 | E1 | G3 |
| G3 | E2 | E3 |
| ... | ... | ... |

In [4]:
# Per facilitare il lavoro in questa prima fase creo un dataframe e poi lo converto in json

hits_array = [[0 for x in range(3)] for y in range(12)]

# 1) disponigo casualmente a coppie di 2 le explanation_human come da consegna
random12 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
for x in range(3):
    for count in range(2): # quota da raggiungere per ogni riga del csv
        random_row = random.choice(random12)
        random12.remove(random_row)
        random_column = random.randint(0, 2)
        
        item = Elemento(fever.loc[x, "id"], fever.loc[x, "statement"], fever.loc[x, "explanation_human"], fever.loc[x, "label"],"G" + str(x+1))
        
        hits_array[random_row][random_column] = item

# 2) riempio le righe restanti con explanation_human casuali
for x in range(6):
    random_row = random.choice(random12)
    random12.remove(random_row)
    random_column = random.randint(0, 2)
    csv_row = random.randint(0, 2)
    
    item = Elemento(fever.loc[csv_row, "id"], fever.loc[csv_row, "statement"], fever.loc[csv_row, "explanation_human"], fever.loc[csv_row, "label"],"G" + str(csv_row+1))
    
    hits_array[random_row][random_column] = item

# 3) in maniera simile al punto 1 inseriamo a gruppi di 3 elementi con explanation_model, assicurandoci che non finiscano nella stessa riga
random12 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
for x in range(3): # riga del csv
    for count in range(3): # quota da raggiungere per ogni riga del csv
        random_row = random.choice(random12)
        random_column = random.randint(0, 2)
        
        if(hits_array[random_row][random_column] == 0 and check_row(hits_array, random_row, x)):
            
            item = Elemento(fever.loc[x, "id"], fever.loc[x, "statement"], fever.loc[x, "explanation_model"], fever.loc[x, "label"],"E" + str(x+1))
            hits_array[random_row][random_column] = item

# 4) riempio gli spazi vuoti con altri elementi con explanation_model, sempre assicurandoci che non finiscano nella stessa riga.
# controllo elemento per elemento, quando troviamo una cella vuota la riempiamo
n = 0
for row in range(12):
    for column in range(3):
        
        if(hits_array[row][column] == 0):
            if(hits_array[row][0] != 0 and hits_array[row][0].type[:1] == 'E'):
                n = int(hits_array[row][0].type[1:]) - 1
            elif(hits_array[row][1] != 0 and hits_array[row][1].type[:1] == 'E'):
                n = int(hits_array[row][1].type[1:]) - 1
            elif(hits_array[row][2] != 0 and hits_array[row][2].type[:1] == 'E'):
                n = int(hits_array[row][2].type[1:]) - 1
            
            csv_row = n 
            while(csv_row == n):
                csv_row = random.randint(0, 2)
            
            item = Elemento(fever.loc[csv_row, "id"], fever.loc[csv_row, "statement"], fever.loc[csv_row, "explanation_model"], fever.loc[csv_row, "label"],"E" + str(csv_row+1))
            hits_array[row][column] = item


In [5]:
# Trasformo in json
hits = []

for i in range(12):
    documents = []
    unit = {
        "unit_id": "unit_" + str(i+1),
        "token_input": id_generator(),
        "token_output": id_generator(),
        "documents_number": 3,
        "documents": []
    }
    for j in range(3):
        document = {
            "id": str(hits_array[i][j].id),
            "statement": hits_array[i][j].statement,
            "explanation": hits_array[i][j].explanation,
            "label": hits_array[i][j].label
        }
        documents.append(document)
    unit["documents"] = documents
    hits.append(unit)

# Salvo il json
serialize_json("hits", "hits.json", hits)


Data serialized to path: hits/hits.json


# Fase preparatoria dei dataframe

In [6]:
# parte di preparazione

# Importazione csv
workers_acl = pandas.read_csv(f'{csv_dir}workers_acl.csv')
workers_answers = pandas.read_csv(f'{csv_dir}workers_answers.csv')
workers_dimensions_selection = pandas.read_csv(f'{csv_dir}workers_dimensions_selection.csv')
workers_info = pandas.read_csv(f'{csv_dir}workers_info.csv')
workers_notes = pandas.read_csv(f'{csv_dir}workers_notes.csv')
workers_questionnaire = pandas.read_csv(f'{csv_dir}workers_questionnaire.csv')
workers_urls = pandas.read_csv(f'{csv_dir}workers_urls.csv')


### Prima di iniziare l'analisi dei dati abbiamo dovuto modellare i dataframe a causa di un numero troppo elevato di dati, dovuto a worker che non hanno completato i task, workers che non hanno inserito il token alla fine del task, worker che avevano fallito i controlli e altre problematiche simili

In [7]:
# Modifichiamo il dataframe workers_answers in modo da mantenere i dati dei workers corretti ed eliminare i dati non necessari.
id_list = ['A2FA30PTJ8OHRA', 'A3GMWT12HZ8BO1', 'A2V54TYNYXD078', 'A338IHJHCEN2DH', 'A32XES1XS1PIYF', 'CLYQ7NB2JJXXGG', 'A2N1GA8PJDDA6P']

new_workers_answers = pandas.DataFrame()

for j in range(len(id_list)):
    worker_rows = workers_answers.loc[workers_answers['worker_id'] == id_list[j]]
    length = len(worker_rows)
    # Rinomino le rows per poter iterare
    rownames = worker_rows.index.values
    for i in range(length):
        rownames[i] = i
    for i in range(length):
        if(worker_rows.loc[i, 'try_last'] == worker_rows.loc[i, 'try_current']):
            new_workers_answers = new_workers_answers.append(worker_rows.loc[i])

# Aggiungiamo manualmente delle righe per problematiche nel csv
worker_rows = workers_answers.loc[workers_answers['worker_id'] == 'A21YKS9HCFVZ9H']
new_workers_answers = new_workers_answers.append(worker_rows.loc[35])
new_workers_answers = new_workers_answers.append(worker_rows.loc[36])
new_workers_answers = new_workers_answers.append(worker_rows.loc[37])
new_workers_answers = new_workers_answers.append(worker_rows.loc[38])
new_workers_answers = new_workers_answers.append(worker_rows.loc[39])
new_workers_answers = new_workers_answers.append(worker_rows.loc[40])
new_workers_answers = new_workers_answers.append(worker_rows.loc[47])
new_workers_answers = new_workers_answers.append(worker_rows.loc[48])
new_workers_answers = new_workers_answers.append(worker_rows.loc[49])

rownames = new_workers_answers.index.values
for i in range(len(new_workers_answers)):
    rownames[i] = i

# In maniera analoga modifichiamo il dataframe workers_notes
new_workers_notes = pandas.DataFrame()

for j in range(len(id_list)):
    worker_rows = workers_notes.loc[workers_notes['worker_id'] == id_list[j]]
    length = len(worker_rows)
    # Rinomino le rows per poter iterare
    rownames = worker_rows.index.values
    for i in range(length):
        rownames[i] = i
    for i in range(length):
        if(worker_rows.loc[i, 'try_last'] == worker_rows.loc[i, 'try_current']):
            new_workers_notes = new_workers_notes.append(worker_rows.loc[i])

# Aggiungiamo manualmente delle righe per problematiche nel csv
worker_rows = workers_notes.loc[workers_notes['worker_id'] == 'A21YKS9HCFVZ9H']
new_workers_notes = new_workers_notes.append(worker_rows.loc[34])
new_workers_notes = new_workers_notes.append(worker_rows.loc[35])
new_workers_notes = new_workers_notes.append(worker_rows.loc[36])
new_workers_notes = new_workers_notes.append(worker_rows.loc[37])
new_workers_notes = new_workers_notes.append(worker_rows.loc[38])
new_workers_notes = new_workers_notes.append(worker_rows.loc[39])
new_workers_notes = new_workers_notes.append(worker_rows.loc[46])
new_workers_notes = new_workers_notes.append(worker_rows.loc[47])
new_workers_notes = new_workers_notes.append(worker_rows.loc[48])

rownames = new_workers_notes.index.values
for i in range(len(new_workers_notes)):
    rownames[i] = i


/var/folders/h7/73m3r7pd4116mzbw6fvwgl740000gn/T/ipykernel_2856/3758238363.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_workers_answers = new_workers_answers.append(worker_rows.loc[i])
/var/folders/h7/73m3r7pd4116mzbw6fvwgl740000gn/T/ipykernel_2856/3758238363.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_workers_answers = new_workers_answers.append(worker_rows.loc[i])
/var/folders/h7/73m3r7pd4116mzbw6fvwgl740000gn/T/ipykernel_2856/3758238363.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_workers_answers = new_workers_answers.append(worker_rows.loc[i])
/var/folders/h7/73m3r7pd4116mzbw6fvwgl740000gn/T/ipykernel_2856/3758238363.py:15: FutureWarning: The frame.append method is deprecated and wil

### Dopo il dowload gli statement, explanation, label e id di alcuni worker non sono stati salvati correttamente in workers_answers, mentre in workers_notes si.
### Grazie alle note in workers_notes possiamo risalire agli elementi corretti e sistemare workers_answers 

In [8]:
# Pulizia elementi problematici
new_workers_notes.loc[0, 'note_text_left'] = ''  #note_text_left aveva come valore NaN

new_workers_notes.loc[1, 'note_text_left'] = new_workers_notes.loc[1, 'note_text_left'][1:] 
new_workers_notes.loc[1, 'note_text_right'] = new_workers_notes.loc[1, 'note_text_right'][:-1] 
new_workers_notes.loc[1, 'note_text_left'] = new_workers_notes.loc[1, 'note_text_left'][:67] + 'ém' + new_workers_notes.loc[1, 'note_text_left'][70:]# è cambiata a causa di passaggio da mac a windows

new_workers_notes.loc[3, 'note_text_left'] = new_workers_notes.loc[3, 'note_text_left'][1:] 
new_workers_notes.loc[3, 'note_text_right'] = new_workers_notes.loc[3, 'note_text_right'][:-1] 

new_workers_notes.loc[4, 'note_text_left'] = '"' + new_workers_notes.loc[4, 'note_text_left'][2:] 
new_workers_notes.loc[4, 'note_text_right'] = new_workers_notes.loc[4, 'note_text_right'][:-2] + '"'

new_workers_notes.loc[5, 'note_text_left'] = fever.loc[1, 'explanation_model'][:279]
new_workers_notes.loc[5, 'note_text_right'] = fever.loc[1, 'explanation_model'][301:]

new_workers_notes.loc[6, 'note_text_left'] = new_workers_notes.loc[6, 'note_text_left'][1:] 
new_workers_notes.loc[6, 'note_text_right'] = new_workers_notes.loc[6, 'note_text_right'][:-1]

new_workers_notes.loc[7, 'note_text_left'] = new_workers_notes.loc[7, 'note_text_left'][1:] 
new_workers_notes.loc[7, 'note_text_right'] = new_workers_notes.loc[7, 'note_text_right'][:-1]
new_workers_notes.loc[7, 'note_text_right'] = new_workers_notes.loc[7, 'note_text_right'][:12] + 'é' + new_workers_notes.loc[7, 'note_text_right'][14:]# è cambiata a causa di passaggio da mac a windows

new_workers_notes.loc[8, 'note_text_left'] = new_workers_notes.loc[8, 'note_text_left'][1:] 
new_workers_notes.loc[8, 'note_text_right'] = new_workers_notes.loc[8, 'note_text_right'][:-1]

new_workers_notes.loc[10, 'note_text_left'] = new_workers_notes.loc[10, 'note_text_left'][1:] 
new_workers_notes.loc[10, 'note_text_right'] = new_workers_notes.loc[10, 'note_text_right'][:-1]

new_workers_notes.loc[13, 'note_text_left'] = new_workers_notes.loc[13, 'note_text_left'][1:] 
new_workers_notes.loc[13, 'note_text_right'] = new_workers_notes.loc[13, 'note_text_right'][:-1]
new_workers_notes.loc[13, 'note_text_raw'] = new_workers_notes.loc[13, 'note_text_raw'][:67]+ 'ém' + new_workers_notes.loc[13, 'note_text_raw'][68:] # è cambiata a causa di passaggio da mac a windows

new_workers_notes.loc[15, 'note_text_left'] = new_workers_notes.loc[15, 'note_text_left'][1:] 
new_workers_notes.loc[15, 'note_text_right'] = new_workers_notes.loc[15, 'note_text_right'][:-1]
new_workers_notes.loc[15, 'note_text_left'] = new_workers_notes.loc[15, 'note_text_left'][:67] + 'ém' + new_workers_notes.loc[15, 'note_text_left'][70:] 

new_workers_notes.loc[26, 'note_text_left'] = new_workers_notes.loc[26, 'note_text_left'][1:] 
new_workers_notes.loc[26, 'note_text_right'] = new_workers_notes.loc[26, 'note_text_right'][:-1]
new_workers_notes.loc[26, 'note_text_raw'] = new_workers_notes.loc[26, 'note_text_raw'][:10] + 'é' + new_workers_notes.loc[26, 'note_text_raw'][10:]

new_workers_notes.loc[27, 'note_text_left'] = new_workers_notes.loc[27, 'note_text_left'][1:] 
new_workers_notes.loc[27, 'note_text_right'] = new_workers_notes.loc[27, 'note_text_right'][:-1] 
new_workers_notes.loc[27, 'note_text_raw'] = new_workers_notes.loc[27, 'note_text_raw'][:-4] + '- ' + new_workers_notes.loc[27, 'note_text_raw'][-4:]# Trattino mancante

new_workers_notes.loc[28, 'note_text_left'] = new_workers_notes.loc[28, 'note_text_left'][1:] 
new_workers_notes.loc[28, 'note_text_right'] = new_workers_notes.loc[28, 'note_text_right'][:-1] 

new_workers_notes.loc[29, 'note_text_left'] = new_workers_notes.loc[29, 'note_text_left'][1:] 
new_workers_notes.loc[29, 'note_text_right'] = new_workers_notes.loc[29, 'note_text_right'][:-1] 

new_workers_notes.loc[30, 'note_text_left'] = new_workers_notes.loc[30, 'note_text_left'][1:] 
new_workers_notes.loc[30, 'note_text_right'] = new_workers_notes.loc[30, 'note_text_right'][:-1] 

new_workers_notes.loc[31, 'note_text_left'] = new_workers_notes.loc[31, 'note_text_left'][1:] 
new_workers_notes.loc[31, 'note_text_right'] = new_workers_notes.loc[31, 'note_text_right'][:-1] 
new_workers_notes.loc[31, 'note_text_left'] = new_workers_notes.loc[31, 'note_text_left'] [:5] + ' ' + new_workers_notes.loc[31, 'note_text_left'][5:8] + ' of ' +  new_workers_notes.loc[31, 'note_text_left'][11:] # Spazi mancanti
new_workers_notes.loc[31, 'note_text_raw'] = new_workers_notes.loc[31, 'note_text_raw'][:3] + ' ' + new_workers_notes.loc[31, 'note_text_raw'][3:]

new_workers_notes.loc[32, 'note_text_left'] = new_workers_notes.loc[32, 'note_text_left'][1:] 
new_workers_notes.loc[32, 'note_text_right'] = new_workers_notes.loc[32, 'note_text_right'][:-1] 

new_workers_notes.loc[33, 'note_text_left'] = new_workers_notes.loc[33, 'note_text_left'][1:] 
new_workers_notes.loc[33, 'note_text_right'] = new_workers_notes.loc[33, 'note_text_right'][:-1] 

new_workers_notes.loc[34, 'note_text_left'] = new_workers_notes.loc[34, 'note_text_left'][1:] 
new_workers_notes.loc[34, 'note_text_right'] = new_workers_notes.loc[34, 'note_text_right'][:-1] 

answers_fix(0, 35, new_workers_answers, new_workers_notes, fever) # Modifichiamo statement, explanation, label e id grazie alle note che abbiamo sistemato


In [9]:
# Salviamo i nuovi dataframe finalmente hanno 36 righe

old_workers_answers = workers_answers
old_workers_notes = workers_notes

workers_answers = new_workers_answers
workers_notes = new_workers_notes

print(f'Lunghezza precedente workers_answers: {len(old_workers_answers)}')
print(f'Lunghezza nuovo workers_answers: {len(workers_answers)}')
print(f'Lunghezza precedente workers_notes: {len(old_workers_notes)}')
print(f'Lunghezza nuovo workers_notes: {len(workers_notes)}')


Lunghezza precedente workers_answers: 85
Lunghezza nuovo workers_answers: 36
Lunghezza precedente workers_notes: 83
Lunghezza nuovo workers_notes: 36


### Infine alcuni valori riguardanti il tempo impiegato dai worker per completare i task erano sostituiti da NaN, pensiamo sia dovuto al tempo minimo da passare su ogni pagina, da noi impostato
### Per questo sostituiamo tutti i valori NaN con il tempo minimo di 30 secondi

In [10]:
for i in range (len(workers_answers)):
    if(math.isnan(workers_answers.loc[i, 'doc_time_elapsed'])):
        workers_answers.loc[i, 'doc_time_elapsed'] = 30.0


# 9.Calcolate il Percent Agreement tra worker per i valori forniti rispetto a ciascuna dimensione categoriale

In [11]:
# Inizializzazioni di Dataframe dove inseriamo le righe del csv originale con lo stesso ID e manipolazione dei df
statement_1 = pandas.DataFrame()
statement_2 = pandas.DataFrame()
statement_3 = pandas.DataFrame()

statement_1 = workers_answers.loc[workers_answers['doc_id'] == 183468]
statement_2 = workers_answers.loc[workers_answers['doc_id'] == 185415]
statement_3 = workers_answers.loc[workers_answers['doc_id'] == 63527]

statement_1_rownames = statement_1.index.values
for i in range(len(statement_1.index)):
    statement_1_rownames[i] = i

statement_2_rownames = statement_2.index.values
for i in range(len(statement_2.index)):
    statement_2_rownames[i] = i

statement_3_rownames = statement_3.index.values
for i in range(len(statement_3.index)):
    statement_3_rownames[i] = i


In [12]:
# Inizializziamo un dataframe dove inserire le explanation in base alla lunghezza percentuale delle note e un dataframe per effettuare i calcoli

percent_agreement = pandas.DataFrame(0, columns = ['Truthfulness 1', 'Truthfulness 2', 'Explanation quality'], index = ['Statement 1', 'Statement 2', 'Statement 3'])

# Sommiamo le lunghezze di tutte le note appartenenti ad una determinata explanation e salvo il risultato in text_percentage
statements = [statement_1, statement_2, statement_3]

for j in range(len(statements)):
    workers_pairs = 0
    for i in range (len(statements[j].index)):
        ground_truth_compare_statement_truth1 = statements[j].loc[i, 'doc_truthfulness1_value']
        ground_truth_compare_statement_truth2 = statements[j].loc[i, 'doc_truthfulness2_value']
        ground_truth_compare_statement_explanation = statements[j].loc[i, 'doc_explanationquality_value']
        n = j+1
        while (n < (len(statements[j].index))):
            confronted_worker_statement_truth1 = statements[j].loc[n, 'doc_truthfulness1_value']
            confronted_worker_statement_truth2 = statements[j].loc[n, 'doc_truthfulness2_value']
            confronted_worker_statement_explanation = statements[j].loc[n, 'doc_explanationquality_value']
            if(ground_truth_compare_statement_truth1 == confronted_worker_statement_truth1):
                percent_agreement.loc[f'Statement {j+1}', 'Truthfulness 1'] += 1
            if(ground_truth_compare_statement_truth2 == confronted_worker_statement_truth2):
                percent_agreement.loc[f'Statement {j+1}', 'Truthfulness 2'] += 1
            in_same_interval(percent_agreement, ground_truth_compare_statement_explanation, confronted_worker_statement_explanation, j+1)
            n += 1
            workers_pairs += 1
    percent_agreement.loc[f'Statement {j+1}', 'Truthfulness 1'] = percent_agreement.loc[f'Statement {j+1}', 'Truthfulness 1'] / workers_pairs
    percent_agreement.loc[f'Statement {j+1}', 'Truthfulness 2'] = percent_agreement.loc[f'Statement {j+1}', 'Truthfulness 2'] / workers_pairs
    percent_agreement.loc[f'Statement {j+1}', 'Explanation quality'] = percent_agreement.loc[f'Statement {j+1}', 'Explanation quality'] / workers_pairs

percent_agreement


,Truthfulness 1,Truthfulness 2,Explanation quality
Statement 1,0.445455,0.436364,0.181818
Statement 2,0.216783,0.237762,0.160839
Statement 3,0.240741,0.370370,0.370370


# 10.Calcolate la percentuale media di testo annotato per ciascuna spiegazione
1. Ordinate le spiegazioni sulla base di tale parametro

In [13]:
# Inizializzazioni di Dataframe dove inseriamo le righe del csv con le stesse explanation
explanation_human_1 = pandas.DataFrame()
explanation_human_2 = pandas.DataFrame()
explanation_human_3 = pandas.DataFrame()
explanation_model_1 = pandas.DataFrame()
explanation_model_2 = pandas.DataFrame()
explanation_model_3 = pandas.DataFrame()

explanation_human_1 = workers_answers.loc[workers_answers['doc_explanation'] == fever.loc[0, 'explanation_human']]
explanation_model_1 = workers_answers.loc[workers_answers['doc_explanation'] == fever.loc[0, 'explanation_model']]
explanation_human_2 = workers_answers.loc[workers_answers['doc_explanation'] == fever.loc[1, 'explanation_human']]
explanation_model_2 = workers_answers.loc[workers_answers['doc_explanation'] == fever.loc[1, 'explanation_model']]
explanation_human_3 = workers_answers.loc[workers_answers['doc_explanation'] == fever.loc[2, 'explanation_human']]
explanation_model_3 = workers_answers.loc[workers_answers['doc_explanation'] == fever.loc[2, 'explanation_model']]


In [14]:
# Inizializziamo un dataframe dove inserire le explanation in base alla lunghezza percentuale delle note e un dataframe per effettuare i calcoli
text_percentage = pandas.DataFrame(0, columns = ['Explanation human 1', 'Explanation human 2', 'Explanation human 3', 'Explanation model 1', 'Explanation model 2', 'Explanation model 3'], index = ['percentage'])

percentage_order = pandas.DataFrame(columns = ['Order', 'Value'])

# Sommiamo le lunghezze di tutte le note appartenenti ad una determinata explanation e salvo il risultato in text_percentage
explanation_humans = [explanation_human_1, explanation_human_2, explanation_human_3]

for j in range(len(explanation_humans)):
    for i in range (len(explanation_humans[j].index)):
        text_percentage.loc['percentage', f'Explanation human {j+1}'] += len(workers_notes.loc[explanation_humans[j].index.values[i], 'note_text_current']) 
    # Otteniamo la media
    text_percentage.loc['percentage', f'Explanation human {j+1}'] /= i+1 
    # Otteniamo la percentuale
    text_percentage.loc['percentage', f'Explanation human {j+1}'] /= (workers_notes.loc[explanation_humans[j].index.values[0], 'note_index_end'] + len(workers_notes.loc[explanation_humans[j].index.values[0], 'note_text_right']))

explanation_models = [explanation_model_1, explanation_model_2, explanation_model_3]

for j in range(len(explanation_models)):
    for i in range (len(explanation_models[j].index)):
        text_percentage.loc['percentage', f'Explanation model {j+1}'] += len(workers_notes.loc[explanation_models[j].index.values[i], 'note_text_current']) 
    # Otteniamo la media
    text_percentage.loc['percentage', f'Explanation model {j+1}'] /= i+1 
    # Otteniamo la percentuale
    text_percentage.loc['percentage', f'Explanation model {j+1}'] /= workers_notes.loc[explanation_models[j].index.values[i], 'note_index_end'] + len(workers_notes.loc[explanation_models[j].index.values[i], 'note_text_right'])

# sommiamo tutte le lunghezze ottenute e uso questo valore per dividere ciascun elemento del dataframe ottenendo la percentuale  
text_percentage /= text_percentage.loc['percentage'].sum()

# troviamo l'elemento con la percentuale maggiore, lo inseriamo nel DataFrame apposito e lo togliamo dal dataframe precedente
for i in range(5):
    percentage_order.loc[i+1, 'Order'] = text_percentage.loc['percentage'].idxmax()
    percentage_order.loc[i+1, 'Value'] = text_percentage.loc['percentage'].max()
    text_percentage = text_percentage.drop(columns = [text_percentage.loc['percentage'].idxmax()])
percentage_order.loc[6, 'Order'] = text_percentage.loc['percentage'].idxmax()
percentage_order.loc[6, 'Value'] = text_percentage.loc['percentage'].max()

percentage_order


,Order,Value
1,Explanation human 3,0.259539
2,Explanation human 1,0.243948
3,Explanation human 2,0.145162
4,Explanation model 1,0.143902
5,Explanation model 3,0.131028
6,Explanation model 2,0.07642


# 11.Determinate il numero di volte in cui i worker aggiornano l’annotazione proposta per ciascuna spiegazione
1. Costruite un ranking delle spiegazioni sulla base di tale parametro

In [15]:
# Inizializziamo un dataframe dove inserire le explanation in base alla lunghezza percentuale delle note e un dataframe per effettuare i calcoli
notes_updated = pandas.DataFrame(0, columns = ['explanation human 1', 'explanation human 2', 'explanation human 3', 'explanation model 1', 'explanation model 2', 'explanation model 3'], index = ['updated notes'])

update_values = pandas.DataFrame(columns = ['Explanation', 'Number of updates'])

# Usando il parametro note_deleted vediamo se le annotazioni sono state selezionate solo una volta
for j in range(len(explanation_humans)):
    for i in range (len(explanation_humans[j].index)):
        if(workers_notes.loc[explanation_humans[j].index.values[i], 'note_deleted'] == False):
            notes_updated.loc['updated notes', f'explanation human {j+1}'] += 1
        else:
            notes_updated.loc['updated notes', f'explanation human {j+1}'] += 2

for j in range(len(explanation_models)):
    for i in range (len(explanation_models[j].index)):
        if(workers_notes.loc[explanation_models[j].index.values[i], 'note_deleted'] == False):
            notes_updated.loc['updated notes', f'explanation model {j+1}'] += 1
        else:
            notes_updated.loc['updated notes', f'explanation model {j+1}'] += 2

# troviamo l'elemento con il numero di update maggiore, lo inseriamo nel DataFrame apposito e lo togliamo dal dataframe precedente
for i in range(5):
    update_values.loc[i+1, 'Explanation'] = notes_updated.loc['updated notes'].idxmax()
    update_values.loc[i+1, 'Number of updates'] = notes_updated.loc['updated notes'].max()
    notes_updated = notes_updated.drop(columns = [notes_updated.loc['updated notes'].idxmax()])
update_values.loc[6, 'Explanation'] = notes_updated.loc['updated notes'].idxmax()
update_values.loc[6, 'Number of updates'] = notes_updated.loc['updated notes'].max()

update_values


,Explanation,Number of updates
1,explanation model 3,17
2,explanation model 2,12
3,explanation human 1,10
4,explanation model 1,10
5,explanation human 2,8
6,explanation human 3,3


# 12.Calcolate il tempo medio impiegato dai worker per valutare ciascun elemento

In [16]:
# Calcoliamo e inseriamo in un dataframe apposito, il tempo medio di risoluzione dei task in base a quale statement viene analizzato
average = pandas.DataFrame(0, columns = ['Statement 1', 'Statement 2', 'Statement 3'], index = ['Average'])

for j in range (len(statements)):
    average_current = 0
    for i in range (len(statements[j])):
        average_current += statements[j].loc[i, 'doc_time_elapsed']        
    
    average_current /= len(statements[j])
    average.loc['Average', f'Statement {j+1}'] = average_current

average


,Statement 1,Statement 2,Statement 3
Average,185.386364,154.372308,149.969167


# 13.Aggiungete eventuali altre analisi che ritenete opportune ed utili

### Abbiamo calcolato e confrontato il punteggio medio di ogni dimensione categoriale di ogni explanation, in modo da poter vedere e confrontare i valori medi assegnati alle explanation, riguardanti gli stessi statement, generati da un umano e quelli generati dalla macchina

In [17]:
# Punteggio medio di ogni dimensione categoriale di ogni explanation
average_explanation_values = pandas.DataFrame(0, columns = ['Explanation human 1', 'Explanation model 1', 'Explanation human 2', 'Explanation model 2', 'Explanation human 3', 'Explanation model 3'], index = ['Truthfulness 1', 'Truthfulness 2', 'Explanation quality'])

average_explanation_quality_values(average_explanation_values, explanation_human_1, 'human 1')
average_explanation_quality_values(average_explanation_values, explanation_model_1, 'model 1')
average_explanation_quality_values(average_explanation_values, explanation_human_2, 'human 2')
average_explanation_quality_values(average_explanation_values, explanation_model_2, 'model 2')
average_explanation_quality_values(average_explanation_values, explanation_human_3, 'human 3')
average_explanation_quality_values(average_explanation_values, explanation_model_3, 'model 3')

average_explanation_values


,Explanation human 1,Explanation model 1,Explanation human 2,Explanation model 2,Explanation human 3,Explanation model 3
Truthfulness 1,3.333333,3.4,0.5,3.888889,1.5,3.1
Truthfulness 2,3.833333,3.8,1.0,3.444444,2.0,3.9
Explanation quality,57.333333,31.4,12.0,84.777778,53.0,84.9


### Abbiamo calcolato la percentuale di testo selezionato per ogni explanation rispetto alla quantità di testo selezionato complessivamente tra tutte le explanation (punto 10 modificato)

In [18]:
# Inizializziamo un dataframe dove inserire le explanation in base alla lunghezza percentuale delle note e un dataframe per effettuare i calcoli
text_percentage = pandas.DataFrame(0, columns = ['Explanation human 1', 'Explanation human 2', 'Explanation human 3', 'Explanation model 1', 'Explanation model 2', 'Explanation model 3'], index = ['percentage'])

percentage_order = pandas.DataFrame(columns = ['Order', 'Value'])

# Sommiamo le lunghezze di tutte le note appartenenti ad una determinata explanation e salvo il risultato in text_percentage
explanation_humans = [explanation_human_1, explanation_human_2, explanation_human_3]

for j in range(len(explanation_humans)):
    for i in range (len(explanation_humans[j].index)):
        text_percentage.loc['percentage', f'Explanation human {j+1}'] += len(workers_notes.loc[explanation_humans[j].index.values[i], 'note_text_current']) 

explanation_models = [explanation_model_1, explanation_model_2, explanation_model_3]

for j in range(len(explanation_models)):
    for i in range (len(explanation_models[j].index)):
        text_percentage.loc['percentage', f'Explanation model {j+1}'] += len(workers_notes.loc[explanation_models[j].index.values[i], 'note_text_current']) 

# sommiamo tutte le lunghezze ottenute e uso questo valore per dividere ciascun elemento del dataframe ottenendo la percentuale  
text_percentage /= text_percentage.loc['percentage'].sum()

# troviamo l'elemento con la percentuale maggiore, lo inseriamo nel DataFrame apposito e lo togliamo dal dataframe precedente
for i in range(5):
    percentage_order.loc[i+1, 'Order'] = text_percentage.loc['percentage'].idxmax()
    percentage_order.loc[i+1, 'Value'] = text_percentage.loc['percentage'].max()
    text_percentage = text_percentage.drop(columns = [text_percentage.loc['percentage'].idxmax()])
percentage_order.loc[6, 'Order'] = text_percentage.loc['percentage'].idxmax()
percentage_order.loc[6, 'Value'] = text_percentage.loc['percentage'].max()

percentage_order


,Order,Value
1,Explanation model 2,0.361621
2,Explanation model 3,0.294807
3,Explanation model 1,0.165294
4,Explanation human 1,0.098797
5,Explanation human 2,0.048132
6,Explanation human 3,0.031349


### Abbiamo calcolato il tempo medio di esecuzione di tutti i task (punto 12 modificato)

In [19]:
# Tempo medio completamento task
average = 0

for i in range (len(workers_answers.index)):
    average += workers_answers.loc[i, 'doc_time_elapsed']

average /= len(workers_answers.index)

average


162.38111111111104